In [1]:
# import some useful libraries
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import metrics

from tqdm.notebook import tqdm
from music21 import converter, instrument, note, chord, stream

In [3]:
glob.glob("../data/piano/*.mid")

['../data/piano\\0fithos.mid',
 '../data/piano\\8.mid',
 '../data/piano\\ahead_on_our_way_piano.mid',
 '../data/piano\\AT.mid',
 '../data/piano\\balamb.mid',
 '../data/piano\\bcm.mid',
 '../data/piano\\BlueStone_LastDungeon.mid',
 '../data/piano\\braska.mid',
 '../data/piano\\caitsith.mid',
 '../data/piano\\Cids.mid',
 '../data/piano\\cosmo.mid',
 '../data/piano\\costadsol.mid',
 '../data/piano\\dayafter.mid',
 '../data/piano\\decisive.mid',
 '../data/piano\\dontbeafraid.mid',
 '../data/piano\\DOS.mid',
 '../data/piano\\electric_de_chocobo.mid',
 '../data/piano\\Eternal_Harvest.mid',
 '../data/piano\\EyesOnMePiano.mid',
 '../data/piano\\ff11_awakening_piano.mid',
 '../data/piano\\ff1battp.mid',
 '../data/piano\\FF3_Battle_(Piano).mid',
 '../data/piano\\FF3_Third_Phase_Final_(Piano).mid',
 '../data/piano\\ff4-airship.mid',
 '../data/piano\\Ff4-BattleLust.mid',
 '../data/piano\\ff4-fight1.mid',
 '../data/piano\\ff4-town.mid',
 '../data/piano\\FF4.mid',
 '../data/piano\\ff4pclov.mid',
 '.

: 

In [2]:
# read in music and prepare it in a list of notes
notes = []
for file in tqdm(glob.glob("../data/piano/*.mid")):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    # Two cases:
    # 1. music file has instrument parts
    # 2. music file has notes in a flat structure
    if parts: 
        notes_to_parse = parts.parts[0].recurse()
    else: 
        notes_to_parse = midi.flat.notes
        
    # each note elements exported and append to an array of notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

  0%|          | 0/92 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
# Data Pre-processing:
# 1. get all pitch name
pitchnames = sorted(set(item for item in notes))

# 2. create a dictionary to map pitches to integers (essentially OHE the pitches)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))


# 3. Creating a input/output sequences for the neural network
# The output for each sequence of length 100 is the first note or chord that come after the sequence of notes
sequence_length = 100
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    
    # map the input/output string(s) to a numerical value for training purposes
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])


# determine the number of different pattern generated
n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
# essentially, the array of array is to be reshape by(x_pattern, y_length, )
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# Post processing on input/outputs
# normalize input
# OHE the output -> so that softmax can be use to generate a probability
# network_input = network_input / len(pitchnames)
# network_output = tf.keras.utils.to_categorical(network_output)

In [13]:
# 4. Load the LSTM model to use in generating a piece of music
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

# Build a model using keras.Sequential.
model = keras.Sequential()

# Input a row of data into an LSTM layer
model.add(keras.layers.LSTM(
    512, 
    input_shape=(network_input.shape[1], network_input.shape[2]), 
    recurrent_dropout=0.3,
    return_sequences=True))


model.add(keras.layers.LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(keras.layers.LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(keras.layers.LSTM(512))
model.add(keras.layers.BatchNormalization())

# dropout ~30% of the first LSTM layer as a regularization technique
# Useful to prevent overfitting
model.add(keras.layers.Dropout(0.3))

# Add some hidden layers/ one more level of dropout layers
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.3))

# output layer with softmax activation on all the 
model.add(keras.layers.Dense(
    units = len(pitchnames),
    activation = "softmax"
))
          
# compile the model 
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

model.load_weights('../data/trainig_outputs/weights-improvement-30-0.3417-bigger.hdf5')


In [14]:
(network_input.shape[1], network_input.shape[2])

(100, 1)

In [27]:
# 5. initialize a random start sequence; generate a series of new notes using that sequence
np.random.seed(97)
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []


# generate n new notes
for note_index in tqdm(range(500)):
    
    # data normalization
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(len(pitchnames))
    
    # model prediction, add to list of pattern
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    
    # pop the first value from the input
    pattern = np.append(pattern, [index])
    pattern = pattern[1:len(pattern)]


  0%|          | 0/500 [00:00<?, ?it/s]

In [28]:
prediction_output

['A3',
 'B-2',
 'B-2',
 'F3',
 'B-3',
 'F3',
 'A2',
 'A2',
 'C3',
 'F2',
 'A3',
 'G2',
 'G2',
 'D3',
 'E-3',
 'G3',
 'D3',
 'D3',
 'A2',
 'E3',
 'A3',
 'B-2',
 'F3',
 'A2',
 'B-3',
 'F3',
 'B-2',
 'C3',
 'E3',
 'F3',
 'C4',
 'E3',
 'G2',
 'D3',
 'G3',
 'D3',
 'A2',
 'E3',
 'A3',
 'B-2',
 'F3',
 'D3',
 'B-3',
 'E3',
 'G3',
 'A2',
 'F3',
 'E3',
 'A3',
 'D3',
 'C3',
 'E3',
 'E-2',
 'B-3',
 'E-2',
 'B-3',
 'E-3',
 'B-3',
 'G3',
 'A3',
 'G3',
 'F#3',
 'E-3',
 'D3',
 'E-3',
 'C3',
 'G3',
 'B-2',
 'A2',
 'G2',
 'F#2',
 'E2',
 'D3',
 'D2',
 'F#3',
 'F#2',
 'D3',
 'F#2',
 'D3',
 'A2',
 'F#3',
 'A2',
 'F#3',
 'D3',
 'A3',
 'D3',
 'A3',
 'D3',
 'E-2',
 'B-3',
 'E-2',
 'B-3',
 'E-3',
 'B-3',
 'G3',
 'A3',
 'G3',
 'F#3',
 'E-3',
 'D3',
 'E-3',
 'C3',
 'G3',
 'B-2',
 'A2',
 'G2',
 'F#2',
 'E2',
 'D3',
 'D2',
 'F#3',
 'D4',
 'F#4',
 'E-4',
 'G4',
 'D4',
 'F#4',
 'C4',
 'E-4',
 'B-3',
 'D4',
 'A3',
 'C4',
 'G3',
 'B-3',
 'F#3',
 'A3',
 'E-3',
 'G3',
 'D3',
 'F#3',
 'D3',
 'G2',
 'D3',
 'G2',
 'D3',
 '

In [29]:
# 6. turning model encoedd output into music
# => decode the pattern into note/chord
# => if chord:
#       decode the cord into an array of notes
#       loop through the array of notes abd create a note object 
#       create chord object that encompasses all notes in the given chord
# => if note:
#       create note object and pitch contained in the pattern
# at each iteration, increase the offset by 0.5

offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [30]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='../data/songs/test_output_06.mid')

'../data/songs/test_output_06.mid'

In [ ]:
# 7. play the generated music
mf = midi.MidiFile()
mf.open('../data/songs/test_output_06.mid')
mf.read()
mf.close()
